In [23]:
import os
import openai
import requests
import re
from gtts import gTTS
from pydub import AudioSegment
from moviepy.editor import ImageClip, AudioFileClip, TextClip, CompositeVideoClip, concatenate_videoclips
from moviepy.video.fx.all import resize
from moviepy.config import change_settings
from moviepy.video import fx as vfx
from textwrap import wrap
import os
from moviepy.editor import *
from moviepy.video.VideoClip import TextClip
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
unsplash_api_key = os.getenv("UNSPLASH_API_KEY")

# Create 'temp' directory if it doesn't exist
if not os.path.exists('temp'):
    os.makedirs('temp')

change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\magick.exe"})

In [24]:
def clean_filename(filename):
    return re.sub('[^A-Za-z0-9 ]+', '', filename)

def generate_text(prompt):
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0.5,
      max_tokens=700
    )
    text = response.choices[0].text.strip()
    print(f"Generated text: {text}")
    return text
    # return prompt

def fetch_image(query):
    response = requests.get(
        'https://api.unsplash.com/photos/random',
        headers={'Authorization': f'Client-ID {unsplash_api_key}'},
        params={'query': query, 'orientation': 'portrait'}
    )
    if response.status_code != 200:  # Check HTTP status code
        print(f"Unsplash API returned status code {response.status_code}. Response: {response.text}")
        return None  # Or handle this situation as you see fit

    try:
        data = response.json()
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON response from Unsplash API. Error: {e}")
        return None  # Or handle this situation as you see fit

    img_url = data['urls']['full']
    img_data = requests.get(img_url).content
    filename = 'temp/' + clean_filename(' '.join(query.split()[:5])) + ".jpg"  # Use only first 5 words of the query
    with open(filename, 'wb') as handler:
        handler.write(img_data)
    print(f"Fetched image for query: {query}")
    return filename

def text_to_speech(text, filename):
    tts = gTTS(text=text, lang='en-us')
    tts.save(filename)
    print(f"Converted text to speech")

def create_video_from_prompt(prompt):
    # Create 'temp' directory if it doesn't exist
    if not os.path.exists('temp'):
        os.makedirs('temp')

    text = generate_text(prompt)
    lines = re.split(r'[.,]', text)

    clips = prepare_clips(lines)

    final_video = concatenate_videoclips(clips, method="compose")
    final_video = vfx.speedx(final_video, factor=1.5)
    final_video.write_videofile("output.mp4", fps=24)
    print(f"Created video")

    # Remove all content from 'temp' directory
    for file in get_temp_files():
        os.remove(file)

    print(f"Removed temporary files")

def prepare_clips(lines):
    clips = []
    temp_files = []

    for line in lines:
        if not line: 
            continue
        line = line.strip()
        image_filename = fetch_image(line)
        if image_filename is None:
            print(f"Skipping line '{line}' due to an error fetching image.")
            continue
        audio_filename = 'temp/' + clean_filename(line[:10]) + ".mp3"
        text_to_speech(line, audio_filename)

        img_clip = ImageClip(image_filename)
        img_clip = resize(img_clip, height=720)  # Resize to standard height
        audio = AudioFileClip(audio_filename)
        audio_duration = audio.duration  # Get audio duration
        img_clip = img_clip.set_duration(audio_duration)  # Set the image duration to match audio

        txt_params = {
            "txt": line,
            "fontsize": 24,
            "font": "Arial-Bold",
            "bg_color": "transparent",
            "color": "white",
            "align": "center",
            "size": (720, None),
            "method": "caption",
            "stroke_color": "black",
        }

        # Create the TextClip object with the given text parameters
        txt_clip = TextClip(**txt_params)

        txt_clip = txt_clip.set_pos('center').set_duration(audio_duration)
        final_clip = CompositeVideoClip([img_clip, txt_clip.set_start(0)]).set_audio(audio)
        clips.append(final_clip)
        temp_files.append(image_filename)
        temp_files.append(audio_filename)
    return clips

def get_temp_files():
    temp_files = []
    for root, dirs, files in os.walk('./temp/'):
        for file in files:
            temp_files.append(os.path.join(root, file))
    return temp_files


In [28]:
humanprompt = "Waterbottle of piss" 
keywords = "water, bottle, piss"

aiprompt = """
Write a marketing poem/jingle for a short format video like a tiktok. Given the prompt : <""" + humanprompt + """>
and the keywords <""" + keywords + """> 
whenever you break a line, or use a fullstop or a comma the content splits, so make it rhyme or use the symbols to establish a pattern or significance, just the script nothing else, no instructions, no directions, no storyboard, no nothing, just the script.
"""
create_video_from_prompt(aiprompt)

Generated text: Verse 1:
On the go, you need water,
A bottle full of piss,
It's the only way, to get through this.

Verse 2:
So don't hesitate, take a sip,
It's the only way to get your fix,
A waterbottle of piss,
Will quench your thirst, no matter what it is.

Chorus:
Waterbottle of piss,
It's the only way to get through this,
So don't hesitate, take a sip,
It's the only way to get your fix.
Unsplash API returned status code 403. Response: Rate Limit Exceeded
Skipping line 'Verse 1:
On the go' due to an error fetching image.
Unsplash API returned status code 403. Response: Rate Limit Exceeded
Skipping line 'you need water' due to an error fetching image.
Unsplash API returned status code 403. Response: Rate Limit Exceeded
Skipping line 'A bottle full of piss' due to an error fetching image.
Unsplash API returned status code 403. Response: Rate Limit Exceeded
Skipping line 'It's the only way' due to an error fetching image.
Unsplash API returned status code 403. Response: Rate Limit Ex

ValueError: max() arg is an empty sequence